## Deploy a model as a web service

## Connect to workspace

In [1]:
import azureml.core
from azureml.workspace import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception unknown locale: UTF-8.


ModuleNotFoundError: No module named 'azureml.workspace'

Run the cell below to train and register a model that predicts the likelihood of a clinic patient being diabetic.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)
# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

First, determine which models have been registered in the workspace

In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

NameError: name 'ws' is not defined

Secondly, specify the model name that we would like to deploy. By default, it will return the latest version if not specified. 

In [3]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

NameError: name 'ws' is not defined

Create folder for web service.

In [4]:
%%writefile $folder_name/score_diabetes.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

#Called when the service is loaded
def init():
    global model
    #get the path to the deployed model file and load it
    model_path=Model.get_model_path('diabetes_model')
    model=joblib.load(model_path)
    
#Called when a request is received
def run(raw_data):
    #get the input data as a numpy array
    data=np.array(json.loads(raw_data['data']))
    #get a prediction from the model
    prediction=model.predict(data)
    #get the correponding classname for each prediction(0 or 1)
    classnames=['not diabetic','diabetic']
    predicted_classes=[]
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    #return the prediction with json
    return json.dumps(predicted_classes)

Writing $folder_name/score_diabetes.py


FileNotFoundError: [Errno 2] No such file or directory: '$folder_name/score_diabetes.py'

The web service will be hold in a container, and the container will need to install any required python packages when it get initialized. In this case we need scikit-learn. We will create a .yml file that tells the container to install this to the environment. 

In [5]:
from azureml.core.cond_dependencies import CondaDependencies
# Add the dependencies for our model 
myenv=CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

ModuleNotFoundError: No module named 'azureml.core.cond_dependencies'

## Ready to deploy
deploy the container as a service and steps are as the following:
1. define an inference configuration, which the scoring and environment files required to load and use the model. 
2. define a deployment configuration in which the service will be host, aka Azure container instance.
3. Deploy model. 
4. Verify status of deployed model. 

In [6]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

NameError: name 'folder_name' is not defined

## Retrieve names of web services in workspace

In [7]:
for webservice_name in ws.webservices:
    print(webservice_name)

NameError: name 'ws' is not defined

## Use the web service
we can use azureml sdk to connect to web service or by making http requests. 

In [8]:
#Connect by azureml sdk
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]


NameError: name 'service' is not defined

In [ ]:
#Connect by HTTP request
endpoint = service.scoring_uri
print(endpoint)

import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

## Authentication
Azure container Instance(ACI) service requires no authentication , however, for production you should consider deploying to Azure Kubernetes Service(AKS) cluster which requires a REST request to include an authentication header.

## Clean up
It will shut down the compute instance.

In [ ]:
service.delete()
print("Service deleted.")